# Toronto emotion speech set (TESS)
#### Ramanan R
#### ramanan.palur@gmail.com
##### -------------------------------------------------------------------

## MFCC feature extraction for TESS and storing it in a dataframe

In [1]:
import pandas as pd # import the necessary modules
import numpy as np 
import os
import librosa
import librosa.feature

def extract_feature(file_name, mfcc, chroma, mel): # function to extract the features
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast') # load the audio file
    if chroma: # if the chroma is true
        stft=np.abs(librosa.stft(X)) # compute the STFT
    result=np.array([]) # create an empty array
    if mfcc: # if the mfcc is true
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0) # compute the MFCCs
        result=np.hstack((result, mfccs)) # concatenate the MFCCs
    if chroma: # if the chroma is true
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0) # compute the chroma features
        result=np.hstack((result, chroma)) # concatenate the chroma features
    if mel: # if the mel is true
        mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0) # compute the mel features
        result=np.hstack((result, mel)) # concatenate the mel features
    return result # return the result

c:\Users\raman\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\raman\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\raman\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os # import the necessary modules
path='E:\\Projects\\Emotion_Detection\\TESS Toronto emotional speech set data\\' # set the path
df=pd.DataFrame(columns=[f'mfcc {i}' for i in range(0,180)]) # create a dataframe with the columns
for file in os.listdir(path): # for each file in the path
    for filename in os.listdir(path+file): # for each file in the path
        f=path+file+"\\"+filename # set the file path
        feature=extract_feature(f, True, True, True) # extract the features
        feature=feature.reshape(1, -1) # reshape the feature 
        df_temp=pd.DataFrame(feature,columns=[f'mfcc {i}' for i in range(len(feature[0]))]) # create a dataframe with the features
        df_temp['label'] = f.split('_')[-1].split('.')[0] # set the label
        df=pd.concat([df, df_temp]) # concatenate the dataframe
df.to_excel('E:\\Projects\\Emotion_Detection\\data.xlsx', index=False) # save the dataframe to an excel file

## Data visualizatiom

In [3]:
print(df['label'].value_counts()) # print the label counts

angry      400
disgust    400
fear       400
happy      400
neutral    400
ps         400
sad        400
Name: label, dtype: int64


In [4]:
df.describe() # print the description of the dataframe

,mfcc 0,mfcc 1,mfcc 2,mfcc 3,mfcc 4,mfcc 5,mfcc 6,mfcc 7,mfcc 8,mfcc 9,...,mfcc 171,mfcc 172,mfcc 173,mfcc 174,mfcc 175,mfcc 176,mfcc 177,mfcc 178,mfcc 179,label
count,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,...,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2.800000e+03,2.800000e+03,2800
unique,2800.000000,2800.000000,2800.000000,2800.000000,2799.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,...,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2800.000000,2.800000e+03,2.800000e+03,7
top,-387.356445,58.005249,-23.070848,-6.495008,-15.636605,12.378984,-29.642843,-2.723384,-13.955122,0.218171,...,0.000275,0.000198,0.000106,0.000062,0.000041,0.000011,0.000003,4.914490e-07,6.342619e-08,angry
freq,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,400


## Splitting the data into training and testing sets

In [5]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.1, random_state=42)

## Random Forest Classification

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
RFmodel = RandomForestClassifier(n_estimators=100, random_state=42)
RFmodel.fit(train_x, train_y)
pred = RFmodel.predict(test_x)
print(classification_report(test_y, pred))
RFaccuracy = accuracy_score(test_y, pred)

              precision    recall  f1-score   support

       angry       1.00      1.00      1.00        42
     disgust       1.00      1.00      1.00        30
        fear       1.00      1.00      1.00        38
       happy       1.00      0.97      0.99        39
     neutral       1.00      1.00      1.00        41
          ps       0.98      1.00      0.99        44
         sad       1.00      1.00      1.00        46

    accuracy                           1.00       280
   macro avg       1.00      1.00      1.00       280
weighted avg       1.00      1.00      1.00       280



## Decision Tree Classification

In [7]:
#import decision tree
from sklearn.tree import DecisionTreeClassifier
DTmodel = DecisionTreeClassifier(random_state=42)
DTmodel.fit(train_x, train_y)
pred = DTmodel.predict(test_x)
print(classification_report(test_y, pred))
DFaccuracy = accuracy_score(test_y, pred)

              precision    recall  f1-score   support

       angry       0.89      0.81      0.85        42
     disgust       0.96      0.87      0.91        30
        fear       0.92      0.89      0.91        38
       happy       0.82      0.85      0.84        39
     neutral       1.00      0.98      0.99        41
          ps       0.85      0.91      0.88        44
         sad       0.90      1.00      0.95        46

    accuracy                           0.90       280
   macro avg       0.91      0.90      0.90       280
weighted avg       0.91      0.90      0.90       280



## Naive bayes

In [8]:
#import naive bayes
from sklearn.naive_bayes import GaussianNB
NBmodel = GaussianNB()
NBmodel.fit(train_x, train_y)
pred = NBmodel.predict(test_x)
print(classification_report(test_y, pred))
NBaccuracy = accuracy_score(test_y, pred)

              precision    recall  f1-score   support

       angry       1.00      0.95      0.98        42
     disgust       0.41      0.83      0.55        30
        fear       0.97      0.92      0.95        38
       happy       1.00      0.51      0.68        39
     neutral       1.00      0.98      0.99        41
          ps       0.75      0.41      0.53        44
         sad       0.78      1.00      0.88        46

    accuracy                           0.80       280
   macro avg       0.84      0.80      0.79       280
weighted avg       0.86      0.80      0.80       280



## Multilayer Perceptron

In [9]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(train_x, train_y)
pred=model.predict(test_x)
print(classification_report(test_y, pred))
MLPaccuracy=accuracy_score(test_y, pred)

              precision    recall  f1-score   support

       angry       1.00      0.98      0.99        42
     disgust       1.00      1.00      1.00        30
        fear       1.00      1.00      1.00        38
       happy       0.97      0.97      0.97        39
     neutral       1.00      1.00      1.00        41
          ps       0.98      1.00      0.99        44
         sad       1.00      1.00      1.00        46

    accuracy                           0.99       280
   macro avg       0.99      0.99      0.99       280
weighted avg       0.99      0.99      0.99       280



## Accuracy analysis

In [10]:
print(f"Random Forest Accuracy: {round(RFaccuracy*100,3)}%")
print(f"Decision Tree Accuracy: {round(DFaccuracy*100,3)}%")
print(f"Naive Bayes Accuracy: {round(NBaccuracy*100,3)}%")
print(f"MLP Accuracy: {round(MLPaccuracy*100,3)}%")


Random Forest Accuracy: 99.643%
Decision Tree Accuracy: 90.357%
Naive Bayes Accuracy: 80.0%
MLP Accuracy: 99.286%
